In [1]:
using JSON

#load OD pair-route incidence
odPairRoute = readall("od_pair_route_incidence_Sioux.json");
odPairRoute = JSON.parse(odPairRoute);

#load link-route incidence
linkRoute = readall("link_route_incidence_Sioux.json");
linkRoute = JSON.parse(linkRoute);

#load OD pair labels
odPairLabel = readall("od_pair_label_dict_Sioux_refined.json");
odPairLabel = JSON.parse(odPairLabel);

odPairLabel_ = readall("od_pair_label_dict__Sioux_refined.json");
odPairLabel_ = JSON.parse(odPairLabel_);

#load link labels
linkLabel = readall("link_label_dict_Sioux.json");
linkLabel = JSON.parse(linkLabel);

linkLabel_ = readall("link_label_dict_Sioux_.json");
linkLabel_ = JSON.parse(linkLabel_);

#load node-link incidence
nodeLink = readall("node_link_incidence_Sioux.json");
nodeLink = JSON.parse(nodeLink);

In [2]:
include("load_network_uni-class.jl")

load_ta_network (generic function with 2 methods)

In [3]:
ta_data = load_ta_network("Sioux Falls");

In [4]:
capacity = ta_data.capacity;
free_flow_time = ta_data.free_flow_time;

In [5]:
############
#Read in the demand file
file = open("../data_original/SiouxFalls_trips.txt")
demands = Dict{(Int64,Int64), Float64}()
s = 0
for line in eachline(file)
    if contains(line, "Origin")
        s = int(split(line)[2])
    else
        pairs = split(line, ";")
        for pair in pairs
            if !contains(pair, "\n")
                pair_vals = split(pair, ":")
                t, demand = int(pair_vals[1]), float(pair_vals[2])
                demands[(s,t)] = demand 
            end
        end
    end
end                
close(file)

In [6]:
demands

Dict{(Int64,Int64),Float64} with 576 entries:
  (23,8)  => 300.0
  (8,23)  => 300.0
  (6,9)   => 400.0
  (18,3)  => 0.0
  (21,13) => 600.0
  (22,16) => 1200.0
  (15,17) => 1500.0
  (22,14) => 1200.0
  (13,9)  => 600.0
  (23,10) => 1800.0
  (24,22) => 1100.0
  (8,9)   => 800.0
  (9,1)   => 500.0
  (17,14) => 700.0
  (16,6)  => 900.0
  (16,12) => 700.0
  (16,13) => 600.0
  (22,2)  => 100.0
  (24,3)  => 0.0
  (7,16)  => 1400.0
  (1,12)  => 200.0
  (4,5)   => 500.0
  (11,8)  => 800.0
  (18,21) => 100.0
  (19,4)  => 200.0
  ⋮       => ⋮

In [7]:
odPairLabel_

Dict{String,Any} with 552 entries:
  "306" => {14,7}
  "153" => {7,16}
  "1"   => {1,2}
  "54"  => {3,9}
  "519" => {23,13}
  "101" => {5,10}
  "371" => {17,3}
  "41"  => {2,19}
  "464" => {21,4}
  "65"  => {3,20}
  "475" => {21,15}
  "447" => {20,10}
  "335" => {15,13}
  "362" => {16,18}
  "505" => {22,23}
  "491" => {22,8}
  "326" => {15,4}
  "543" => {24,14}
  "299" => {13,24}
  "168" => {8,7}
  "403" => {18,12}
  "159" => {7,22}
  "228" => {10,22}
  "332" => {15,10}
  "190" => {9,6}
  ⋮     => ⋮

In [8]:
demandsVec = zeros(552)

for i = 1:length(demandsVec)
    demandsVec[i] = demands[(odPairLabel_["$i"][1], odPairLabel_["$i"][2])]
end

In [9]:
demandsVec

552-element Array{Float64,1}:
  100.0
  100.0
  500.0
  200.0
  300.0
  500.0
  800.0
  500.0
 1300.0
  500.0
  200.0
  500.0
  300.0
    ⋮  
  500.0
  700.0
  400.0
  400.0
  300.0
  300.0
    0.0
  100.0
  400.0
  500.0
 1100.0
  700.0

In [17]:
for key=keys(odPairRoute)
    if contains(key, "552-")
        println(key)
    end
end

552-7117
552-7118
552-7119
552-7123
552-7122
552-7124
552-7121
552-7120


In [19]:
i = 552
j = 7117
"$(i)-$(j)" in keys(odPairRoute)

true

In [24]:
using JuMP
using Gurobi

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()


@defVar(m, linkFlow[1:76])
@defVar(m, pathFlow[1:7124])

pathFlowSum = Dict()

for i=1:552
    pathFlowSum[i] = 0
    for j=1:7124
        if "$(i)-$(j)" in keys(odPairRoute)
            pathFlowSum[i] += pathFlow[j]
        end
    end
    if i==552
        println(pathFlowSum[i])
#         assert(1==2)
    end
    @addConstraint(m, pathFlowSum[i] == demandsVec[i])
end

pathFlowLinkSum = Dict()

for a=1:76
    pathFlowLinkSum[a] = 0
    for j=1:7124
        if "$(a)-$(j)" in keys(linkRoute)
            pathFlowLinkSum[a] += pathFlow[j]
        end
    end
    if a==1
        println(pathFlowLinkSum[a])
        assert(1==2)
    end
    @addConstraint(m, pathFlowLinkSum[a] == linkFlow[a])
end

for j=1:7124
    @addConstraint(m, pathFlow[j] >= 0)
end

@defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:76})

@setNLObjective(m, Min, f)

solve(m)

pathFlow[7117] + pathFlow[7118] + pathFlow[7119] + pathFlow[7120] + pathFlow[7121] + pathFlow[7122] + pathFlow[7123] + pathFlow[7124]
pathFlow[1] + pathFlow[4] + pathFlow[5] + pathFlow[6] + pathFlow[9] + pathFlow[10] + pathFlow[15] + pathFlow[19] + pathFlow[20] + pathFlow[21] + pathFlow[24] + pathFlow[25] + pathFlow[32] + pathFlow[33] + pathFlow[34] + pathFlow[40] + pathFlow[41] + pathFlow[42] + pathFlow[43] + pathFlow[44] + pathFlow[50] + pathFlow[51] + pathFlow[52] + pathFlow[53] + pathFlow[57] + pathFlow[58] + pathFlow[63] + pathFlow[69] + pathFlow[70] + pathFlow[71] + pathFlow[77] + pathFlow[78] + pathFlow[79] + pathFlow[80] + pathFlow[81] + pathFlow[89] + pathFlow[90] + pathFlow[91] + pathFlow[92] + pathFlow[98] + pathFlow[99] + pathFlow[102] + pathFlow[109] + pathFlow[110] + pathFlow[111] + pathFlow[119] + pathFlow[268] + pathFlow[273] + pathFlow[276] + pathFlow[277] + pathFlow[282] + pathFlow[292] + pathFlow[301] + pathFlow[302] + pathFlow[314] + pathFlow[315] + pathFlow[325] + 

LoadError: assertion failed
while loading In[24], in expression starting on line 29

In [13]:
getValue(linkFlow)

76-element Array{Float64,1}:
 26873.3 
  8587.79
 41331.9 
  5812.21
 36381.3 
 14518.4 
 22632.7 
 12144.9 
 18618.4 
 11044.2 
 16489.1 
  8475.87
 18808.8 
     ⋮   
  7664.24
  9768.96
 17811.4 
  7000.0 
  7536.99
 10184.8 
  8248.59
 10271.0 
  9824.83
 12493.8 
  9292.07
  8740.61

In [14]:
getObjectiveValue(m)

5.574336405569012e6

In [15]:
1 != 2

true

In [15]:
using JuMP
using Gurobi

# m = Model(solver=GurobiSolver(OutputFlag=false))
m = Model()

numLinks = 76  # number of links
numNodes = 24  # number of nodes

@defVar(m, linkFlow[1:numLinks])

@defVar(m, odLinkFlow[keys(demands), 1:numLinks])
for (s,t) = keys(demands)
    for k = 1:numLinks
        @addConstraint(m, odLinkFlow[(s,t), k] >= 0)
    end
end

for k = 1:numLinks
    linkFlowPartial = sum([odLinkFlow[(s,t), k] for (s,t) = keys(demands)])
    @addConstraint(m, linkFlowPartial == linkFlow[k])
end

for l = 1:numNodes
    for (s,t) = keys(demands)
        if s != t
            odLinkFlowPartial = sum([nodeLink["$(l-1)-$(k-1)"] * odLinkFlow[(s,t), k] for k = 1:numLinks])
            if (l == s)
                @addConstraint(m, odLinkFlowPartial + demands[(s,t)] == 0)
            elseif (l == t)
                @addConstraint(m, odLinkFlowPartial - demands[(s,t)] == 0)
            else
                @addConstraint(m, odLinkFlowPartial == 0)
            end
        end
    end
end

@defNLExpr(f, sum{free_flow_time[a]*linkFlow[a] + .03*free_flow_time[a]*((linkFlow[a])^5)/((capacity[a])^4), a = 1:76})

@setNLObjective(m, Min, f)

solve(m)

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   127756
Number of nonzeros in inequality constraint Jacobian.:    43776
Number of nonzeros in Lagrangian Hessian.............:       76

Total number of variables............................:    43852
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:    13324
Total number of inequality constraints...............:    43776
        inequality constraints with only lower bounds:    43776
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

:Optimal

In [16]:
getValue(linkFlow)

76-element Array{Float64,1}:
  4519.08
  8094.66
  4494.66
  5991.76
  8119.08
 14030.6 
  9973.71
 14006.4 
 18030.6 
  5300.0 
 18006.4 
  8806.5 
 15796.7 
     ⋮   
  8607.39
 10259.5 
 18409.9 
  7000.0 
  8619.54
  9626.21
  8400.44
  9661.82
  7861.83
 11121.4 
 10309.4 
  7902.98

In [17]:
getObjectiveValue(m)

4.231335283350912e6